In [4]:

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

**Load Metadata**

In [5]:
# Load metadata
metadata_path = '/kaggle/input/isic-2024-challenge/train-metadata.csv'
metadata = pd.read_csv(metadata_path)

# Print the first few rows of the DataFrame to check if data is loaded correctly
metadata.head()



/tmp/ipykernel_13/1363166166.py:3: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_path)


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


**Image Batch Generate**

In [6]:
# Print column names to verify the presence of 'isic_id'
print(metadata.columns)


Index(['isic_id', 'target', 'patient_id', 'age_approx', 'sex',
       'anatom_site_general', 'clin_size_long_diam_mm', 'image_type',
       'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
       'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
       'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id',
       'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
       'mel_mitotic_index', '

**Define the Model**

In [7]:
# Check for missing or empty values in 'isic_id' column
missing_ids = metadata['isic_id'].isnull().sum()
empty_ids = (metadata['isic_id'].str.strip() == '').sum()
print(f"Missing 'isic_id' values: {missing_ids}")
print(f"Empty 'isic_id' values: {empty_ids}")

Missing 'isic_id' values: 0
Empty 'isic_id' values: 0


In [8]:
import os
# Append .jpg to each filename in the 'isic_id' column
metadata['isic_id'] = metadata['isic_id'].apply(lambda x: f"{x}.jpg")

# Check if the image files exist
metadata['file_exists'] = metadata['isic_id'].apply(lambda x: os.path.isfile(os.path.join('/kaggle/input/isic-2024-challenge/train-image/image', x)))
print(metadata['file_exists'].value_counts())

# Filter out rows where the image file does not exist
metadata = metadata[metadata['file_exists']]

# If no files exist, raise an error
if metadata.empty:
    raise ValueError("No valid image files found. Please check the filenames and directory.")

file_exists
True    401059
Name: count, dtype: int64


In [9]:
# Create the 'label' column
if 'tbp_lv_nevi_confidence' in metadata.columns:
    metadata['label'] = metadata['tbp_lv_nevi_confidence'].apply(lambda x: 1 if x > 0.5 else 0)
else:
    raise ValueError("The necessary column for labels is missing from the metadata.")

In [10]:
# Create train and validation splits
train_df, val_df = train_test_split(metadata, test_size=0.2, random_state=0)


In [11]:
train_df['label'] = train_df['label'].astype(str)
val_df['label'] = val_df['label'].astype(str)

In [12]:
# Convert labels to strings
train_df['label'] = train_df['label'].astype(str)
val_df['label'] = val_df['label'].astype(str)

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory='/kaggle/input/isic-2024-challenge/train-image/image',
    x_col='isic_id',  # Adjust if your image filenames are in a different column
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    directory='/kaggle/input/isic-2024-challenge/train-image/image',
    x_col='isic_id',  # Adjust if your image filenames are in a different column
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 320847 validated image filenames belonging to 2 classes.
Found 80212 validated image filenames belonging to 2 classes.


In [13]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [14]:
# Load the EfficientNetB0 model with pre-trained weights, without the top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

I0000 00:00:1724752714.074294      13 service.cc:145] XLA service 0x57c93db3a230 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724752714.074357      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1724752714.074362      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1724752714.074365      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1724752714.074368      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1724752714.074371      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1724752714.074373      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1724752714.074376      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1724752714.074379      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [15]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [16]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
# Set up callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', verbose=1)

In [19]:
print("Training data directory:", train_generator.directory)
print("Validation data directory:", val_generator.directory)


Training data directory: /kaggle/input/isic-2024-challenge/train-image/image
Validation data directory: /kaggle/input/isic-2024-challenge/train-image/image


In [20]:
train_datagen_simple = ImageDataGenerator(rescale=1./255)
train_generator_simple = train_datagen_simple.flow_from_dataframe(
    train_df,
    directory='/kaggle/input/isic-2024-challenge/train-image/image',
    x_col='isic_id',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator_simple = val_datagen.flow_from_dataframe(
    val_df,
    directory='/kaggle/input/isic-2024-challenge/train-image/image',
    x_col='isic_id',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Train with the simplified generators
history = model.fit(
    train_generator_simple,
    steps_per_epoch=train_generator_simple.samples // train_generator_simple.batch_size,
    validation_data=val_generator_simple,
    validation_steps=val_generator_simple.samples // val_generator_simple.batch_size,
    epochs=5,
    verbose=2
)


Found 320847 validated image filenames belonging to 2 classes.
Found 80212 validated image filenames belonging to 2 classes.
Epoch 1/5


/usr/local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/local/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/local/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/local/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in dispatch_qu

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=5,  # Reduced for testing
    verbose=2
)


In [ ]:
val_loss, val_accuracy = model.evaluate(val_generator, steps=val_generator.samples // val_generator.batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")